In [23]:
import matplotlib.pyplot as plt
import numpy as np

In [24]:
import pandas as pd
from os import path, getcwd


In [25]:
base_dir = path.abspath(path.join(getcwd(), '..'))
df = pd.read_csv(path.join(base_dir, "tmp","owid-covid-data.csv"), parse_dates=["date"])

In [26]:
duplicates = df[df.duplicated(subset=["iso_code", "date"], keep=False)]
print(f"\n🔍 Duplicate records by iso_code + date: {len(duplicates)}")
if not duplicates.empty:
    print(duplicates.head())


🔍 Duplicate records by iso_code + date: 15540
       iso_code continent    location       date  total_cases  new_cases  \
102540      TLS      Asia  East Timor 2021-02-12         80.0        0.0   
102541      TLS      Asia  East Timor 2021-02-13         80.0        0.0   
102542      TLS      Asia  East Timor 2021-02-14        102.0       22.0   
102543      TLS      Asia  East Timor 2021-02-15        102.0        0.0   
102544      TLS      Asia  East Timor 2021-02-16        102.0        0.0   

        new_cases_smoothed  total_deaths  new_deaths  new_deaths_smoothed  \
102540               1.429           0.0         0.0                  0.0   
102541               1.429           0.0         0.0                  0.0   
102542               3.143           0.0         0.0                  0.0   
102543               3.143           0.0         0.0                  0.0   
102544               3.143           0.0         0.0                  0.0   

        ...  male_smokers  handwa

In [27]:
df["calc_total"] = df.groupby("iso_code")["new_cases"].cumsum()
inconsistent = df[
    (df["total_cases"] <= df["calc_total"]) & 
    (df["total_cases"].notnull())
]

print(f"\n⚠️ Rows where total_cases < sum(new_cases): {len(inconsistent)}")
if not inconsistent.empty:
    print(inconsistent[["iso_code", "location", "date", "total_cases", "calc_total"]].head())


⚠️ Rows where total_cases < sum(new_cases): 410159
  iso_code     location       date  total_cases  calc_total
0      AFG  Afghanistan 2020-01-05          0.0         0.0
1      AFG  Afghanistan 2020-01-06          0.0         0.0
2      AFG  Afghanistan 2020-01-07          0.0         0.0
3      AFG  Afghanistan 2020-01-08          0.0         0.0
4      AFG  Afghanistan 2020-01-09          0.0         0.0


In [28]:
df["z_score"] = df.groupby("iso_code")["new_cases"]\
    .transform(lambda x: (x - x.mean()) / x.std(ddof=0))

outliers = df[df["z_score"] > 10]  # You can adjust the threshold
print(f"\n🚨 Outliers detected (Z-score > 10): {len(outliers)}")
print(outliers[["iso_code", "location", "date", "new_cases", "z_score"]].head())


🚨 Outliers detected (Z-score > 10): 962
    iso_code     location       date  new_cases    z_score
518      AFG  Afghanistan 2021-06-06     8463.0  10.711177
525      AFG  Afghanistan 2021-06-13    10637.0  13.509150
532      AFG  Afghanistan 2021-06-20    12045.0  15.321269
539      AFG  Afghanistan 2021-06-27    12314.0  15.667476
546      AFG  Afghanistan 2021-07-04    10528.0  13.368865


In [29]:
df["new_cases_vs_pop"] = df["new_cases"] / df["population"]
absurd_cases = df[df["new_cases_vs_pop"] > 1]
print(f"\n❗️ Rows where new_cases > population: {len(absurd_cases)}")
print(absurd_cases[["iso_code", "location", "date", "new_cases", "population"]].head())


❗️ Rows where new_cases > population: 0
Empty DataFrame
Columns: [iso_code, location, date, new_cases, population]
Index: []


In [30]:

# Z-score por país (iso_code)
df['z_score'] = df.groupby('iso_code')['new_cases'].transform(
    lambda x: (x - x.mean()) / x.std(ddof=0)
)

# Outliers com z-score acima de 5 (ajustável)
outliers_z = df[df['z_score'].abs() > 5]

print(f"🔍 Found {len(outliers_z)} extreme new_cases (z-score > 5):")
print(outliers_z[['iso_code', 'date', 'new_cases', 'z_score']].sort_values('z_score', ascending=False))


🔍 Found 3061 extreme new_cases (z-score > 5):
        iso_code       date  new_cases    z_score
305296       PCN 2022-07-24        4.0  40.902323
422069       WLF 2022-10-16     2654.0  40.467486
377907       TZA 2021-08-08    20607.0  40.013282
415121       VAT 2020-10-18       14.0  38.436387
322099       SHN 2022-09-25     1298.0  38.331501
...          ...        ...        ...        ...
344550       SLE 2021-01-17      167.0   5.006461
55564        BGR 2020-11-08    21641.0   5.005529
3978         ALB 2021-09-26     5366.0   5.004969
277556  OWID_NAM 2021-01-10  1884976.0   5.004056
82649        CRI 2021-09-05    16531.0   5.002171

[3061 rows x 4 columns]


In [31]:
total_eq_zero = df[df["total_cases"] == 0]

In [32]:
top10 = (
    df[df['new_cases'].notna() & df['population'].notna()]
    .assign(cases_pct=lambda x: x['new_cases'] / x['population'])
    .sort_values('cases_pct', ascending=False)
    .loc[:, ['iso_code', 'location', 'date', 'new_cases', 'population', 'cases_pct']]
    .head(10)
)


In [33]:
top10

,iso_code,location,date,new_cases,population,cases_pct
322099,SHN,Saint Helena,2022-09-25,1298.0,5401,0.240326
422069,WLF,Wallis and Futuna,2022-10-16,2654.0,11596,0.228872
236283,MHL,Marshall Islands,2022-08-21,9086.0,41593,0.218450
124532,FLK,Falkland Islands,2022-05-15,734.0,3801,0.193107
263015,NRU,Nauru,2022-06-26,2055.0,12691,0.161926
124546,FLK,Falkland Islands,2022-05-29,537.0,3801,0.141279
263022,NRU,Nauru,2022-07-03,1565.0,12691,0.123316
236290,MHL,Marshall Islands,2022-08-28,4885.0,41593,0.117448
328613,SPM,Saint Pierre and Miquelon,2022-03-27,577.0,5885,0.098046
396124,TUV,Tuvalu,2022-11-13,1096.0,11335,0.096692


In [42]:
import pandas as pd

# Supondo que df já tenha 'country', 'date', 'new_cases', 'population' e 'cases_pct'
df['cases_pct'] = df['new_cases'] / df['population']

# Define as faixas populacionais
bins = [10, 10_000, 100_000, 1_000_000, 10_000_000, 100_000_000, 1_000_000_000, 10_000_000_000]
labels = [
    "10–10k", "10k–100k", "100k–1M", "1M–10M",
    "10M–100M", "100M–1B", "1B–10B"
]
df['population_range'] = pd.cut(df['population'], bins=bins, labels=labels, right=False)

# Remove países com população nula
df_filtered = df[df['population'].notna() & df['new_cases'].notna()]

# Agrupa e calcula
result = (
    df_filtered.sort_values('cases_pct', ascending=False)
    .groupby('population_range')
    .agg(
        top_1_pct=('cases_pct', 'max'),
        top_1_country=('location', lambda x: x.iloc[0]),
        top_1_date=('date', lambda x: x.iloc[0]),
        avg_pct=('cases_pct', 'mean'),
        n_entries=('cases_pct', 'count')
    )
    .reset_index()
)

# Mostra o resultado formatado
import pandas as pd
import numpy as np

# Formata como porcentagem
result['top_1_pct'] = (result['top_1_pct'] * 100).round(2).astype(str) + '%'
result['avg_pct'] = (result['avg_pct'] * 100).round(2).astype(str) + '%'

C:\Users\lucas\AppData\Local\Temp\ipykernel_7716\3103668013.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('population_range')


In [43]:
result

,population_range,top_1_pct,top_1_country,top_1_date,avg_pct,n_entries
0,10–10k,24.03%,Saint Helena,2022-09-25,0.02%,13392
1,10k–100k,22.89%,Wallis and Futuna,2022-10-16,0.02%,50217
2,100k–1M,6.55%,Brunei,2022-03-06,0.02%,63606
3,1M–10M,5.48%,Denmark,2022-02-13,0.01%,110481
4,10M–100M,5.24%,South Korea,2022-03-20,0.01%,131056
5,100M–1B,1.97%,European Union (27),2022-01-30,0.01%,28015
6,1B–10B,2.84%,China,2022-12-25,0.01%,13392
